# Тестирование метода Такахаси-Таками для расчета СМО М/H2/n

Идея этого алгоритма была впервые предложена Такахаси и Таками

>Takahashi Y., Takami Y. A Numerical Method for the Steady-State
Probabilities of a GI/G/c Queuing System in a General Class // J. of the
Operat. Res. Soc. of Japan. 1976. — v. 19, no. 2. — P. 147–157.

Позволяет рассчитать систему при произвольном коэффициенте вариации времени обслуживания. При коэффициенте вариации времени обслуживания < 1 параметры аппроксимирующего Н2-распределения являются комплексными, что не мешает получению осмысленных результатов

Для верификации используется имитационное моделирование (ИМ)


#### Импорт необходимых модулей

In [29]:
from most_queue.sim.qs_sim import QueueingSystemSimulator
from most_queue.sim import rand_distribution as rd
from most_queue.theory.mgn_tt import MGnCalc
from most_queue.general_utils.tables import probs_print, times_print

import time
import math


#### Задаем входные параметры

In [30]:
n = 3  # число каналов
l = 1.0  # интенсивность вх потока
ro = 0.8  # коэффициент загрузки
b1 = n * ro / l  # среднее время обслуживания
num_of_jobs = 800000  # число требуемых к обслуживанию заявок (для ИМ)
b_coev = [0.42, 1.5]  # коэффициенты вариации времени обслуживания

#### Запуск расчета и ИМ для значений коэфф вариации времени обсл (0.42 и 1.5):

In [31]:
for k in range(len(b_coev)):
    #  расчет начальных моментов времени обслуживания по заданному среднему и коэфф вариации
    b = [0.0] * 3
    alpha = 1 / (b_coev[k] ** 2)
    b[0] = b1
    b[1] = math.pow(b[0], 2) * (math.pow(b_coev[k], 2) + 1)
    b[2] = b[1] * b[0] * (1.0 + 2 / alpha)

    #  запуск метода Такахаси-Таками
    tt_start = time.process_time()

    tt = MGnCalc(n, l, b)
    tt.run()
    # получение результатов численных расчетов
    p_tt = tt.get_p()
    v_tt = tt.get_v()
    tt_time = time.process_time() - tt_start
    # также можно узнать сколько итераций потребовалось
    num_of_iter = tt.num_of_iter_

    # запуск ИМ для верификации результатов
    im_start = time.process_time()

    qs = QueueingSystemSimulator(n)

    # Задаем вх поток заявок. М - экспоненциальный с интенсивностью l
    qs.set_sources(l, 'M')

    # Задаем параметры каналов обслуживания Гамма-распределением.
    # Параметры распределения подбираем с помощью метода библиотеки random_distribution
    gamma_params = rd.Gamma.get_mu_alpha([b[0], b[1]])
    qs.set_servers(gamma_params, 'Gamma')

    # Запуск ИМ
    qs.run(num_of_jobs)

    # Получение результатов
    p = qs.get_p()
    v_sim = qs.v
    im_time = time.process_time() - im_start

    # Вывод результатов расчета

    print("\nСравнение результатов расчета методом Такахаси-Таками и ИМ.\n"
          f"ИМ - M/Gamma/{n:^2d}\nТакахаси-Таками - M/H2/{n:^2d}"
          "с комплексными параметрами\n"
          f"Коэффициент загрузки: {ro:^1.2f}\n"
          f"Коэффициент вариации времени обслуживания: {b_coev[k]:^1.2f}\n"
          f"Количество итераций алгоритма Такахаси-Таками: {num_of_iter:^4d}\n"
          f"Время работы алгоритма Такахаси-Таками: {tt_time:^5.3f} c\n"
          f"Время работы ИМ: {im_time:^5.3f} c")
    probs_print(p, p_tt, 10)
    times_print(v_sim, v_tt, is_w=False)

Start simulation



100%|██████████| 800000/800000 [00:02<00:00, 291569.44it/s]


Simulation is finished


Сравнение результатов расчета методом Такахаси-Таками и ИМ.
ИМ - M/Gamma/3 
Такахаси-Таками - M/H2/3 с комплексными параметрами
Коэффициент загрузки: 0.80
Коэффициент вариации времени обслуживания: 0.42
Количество итераций алгоритма Такахаси-Таками:  10 
Время работы алгоритма Такахаси-Таками: 0.000 c
Время работы ИМ: 1.391 c
------------------------------------
       Вероятности состояний        
------------------------------------
 №  |     Числ      |      ИМ       
------------------------------------
 0  |   0.053201    |   0.052449    
 1  |    0.13317    |    0.1333     
 2  |    0.17406    |    0.1772     
 3  |    0.17103    |    0.1687     
 4  |     0.136     |    0.13599    
 5  |   0.099993    |    0.10108    
 6  |   0.071056    |    0.07157    
 7  |    0.04971    |   0.048836    
 8  |   0.034526    |   0.033787    
 9  |   0.023899    |   0.023674    
------------------------------------

Начальные моменты времени пребывания заявок в системе:

100%|██████████| 800000/800000 [00:02<00:00, 294932.81it/s]


Simulation is finished


Сравнение результатов расчета методом Такахаси-Таками и ИМ.
ИМ - M/Gamma/3 
Такахаси-Таками - M/H2/3 с комплексными параметрами
Коэффициент загрузки: 0.80
Коэффициент вариации времени обслуживания: 1.50
Количество итераций алгоритма Такахаси-Таками:  26 
Время работы алгоритма Такахаси-Таками: 0.016 c
Время работы ИМ: 0.984 c
------------------------------------
       Вероятности состояний        
------------------------------------
 №  |     Числ      |      ИМ       
------------------------------------
 0  |   0.058345    |   0.056182    
 1  |    0.13622    |    0.13394    
 2  |    0.15252    |    0.15317    
 3  |    0.10273    |    0.09595    
 4  |    0.07841    |   0.077311    
 5  |   0.064082    |   0.064823    
 6  |   0.054151    |   0.055108    
 7  |   0.046478    |   0.047882    
 8  |   0.040177    |   0.040389    
 9  |   0.034845    |   0.034815    
------------------------------------

Начальные моменты времени пребывания заявок в системе: